# logger (optional)
Run set_logger() only once. If the output becomes incorrect, please restart the kernel.

In [ ]:
####
#### when you change set_logger(), you need to restart the kernel
####
from cryspy.util.utility import set_logger
set_logger()
#set_logger(noprint=True, logfile='log_cryspy', errfile='err_cryspy')    # write log and err messages to files

# Random structure generation
## with pyxtal

In [ ]:
from cryspy.RS.gen_struc_RS import gen_pyxtal

In [ ]:
nstruc = 10
atype = ('Na', 'Cl')
nat = (4, 4)
mindist = ((2.0, 1.5),
           (1.5, 2.0))
spgnum = 'all'

In [ ]:
init_struc_data = gen_pyxtal.gen_struc(
    nstruc=nstruc,
    atype=atype,
    nat=nat,
    mindist=mindist,
    spgnum=spgnum,
)

In [ ]:
init_struc_data.keys()

# Evolutionary algorithm

## Situation: parent A (, parent B) --> child

Prepare two parent structures as pymatgen Structure object.  
In this example, just use the results of RS for Cu4Au4.

In [ ]:
import pickle
with open('./Cu4Au4_sample/opt_struc_data.pkl', 'rb') as f:
    opt_struc_data = pickle.load(f)

### Crossover

In [ ]:
from cryspy.EA.gen_struc_EA import crossover

In [ ]:
# you can change parent_A and parent_B
parent_A = opt_struc_data[0]
parent_B = opt_struc_data[1]

In [ ]:
atype = ('Cu', 'Au')
nat = (4, 4)
mindist = ((1.5, 1.5),
           (1.5, 1.5))

In [ ]:
child = crossover.gen_child(
    atype=atype,
    nat=nat,
    mindist=mindist,
    parent_A=parent_A,
    parent_B=parent_B,
)

In [ ]:
child    # to print in jupyter

### Permutation

In [ ]:
from cryspy.EA.gen_struc_EA import permutation

In [ ]:
# you can change parent_A
parent_A = opt_struc_data[0]

In [ ]:
atype = ('Cu', 'Au')
nat = (4, 4)
mindist = ((1.5, 1.5),
           (1.5, 1.5))
ntimes = 1    # number of times to perform permutation

In [ ]:
child = permutation.gen_child(
    atype=atype,
    mindist=mindist,
    parent_A=parent_A,
    ntimes=ntimes,
)

### Strain

In [ ]:
from cryspy.EA.gen_struc_EA import strain

In [ ]:
# you can change parent_A
parent_A = opt_struc_data[0]

In [ ]:
atype = ('Cu', 'Au')
nat = (4, 4)
mindist = ((1.5, 1.5),
           (1.5, 1.5))
sigma_st = 0.05    # standard deviation of strain

In [ ]:
child = strain.gen_child(
    atype=atype,
    mindist=mindist,
    parent_A=parent_A,
    sigma_st=sigma_st,
)

In [ ]:
child

## Situation: parent group, fitness --> children

Prepare structure and fitness (energy) data as dict.  
The key is structure ID.  
In this example, just use the results of RS for Cu4Au4.

e.g.  
struc_data = {0: (pymatgen Structure), 1: (pymatgen Structure), ...}  
fitness = {0: 0.019632287242441926, 1: -0.005437509701440302, ...}

In [ ]:
import pickle
with open('./Cu4Au4_sample/opt_struc_data.pkl', 'rb') as f:
    opt_struc_data = pickle.load(f)
with open('./Cu4Au4_sample/rslt_data.pkl', 'rb') as f:
    rslt_data = pickle.load(f)

In [ ]:
struc_data = opt_struc_data    # dict
fitness = rslt_data['E_eV_atom'].to_dict()    # you may include None or np.nan for values

In [ ]:
fitness    # to print in jupyter

Survival of the fittest

In [ ]:
from cryspy.EA.survival import survival_fittest
from cryspy.EA.gen_struc_EA.select_parents import SelectParents
from cryspy.EA.gen_struc_EA import crossover, permutation, strain

In [ ]:
n_fittest = 5    # number of survivors

In [ ]:
ranking, _, _ = survival_fittest(
    fitness=fitness,
    struc_data=struc_data,
    elite_struc=None,
    elite_fitness=None,
    n_fittest=n_fittest,
    fit_reverse=False,
    emax_ea=None,
    emin_ea=None,
)

ranking <-- without structure duplication

In [ ]:
ranking    # to print in jupyter

SelectParents class

In [ ]:
sp = SelectParents(ranking)    # after set_xxx, we can use sp.get_parents(n_parent)
sp.set_tournament(t_size=2)

Generate children by crossover

In [ ]:
atype = ('Cu', 'Au')
nat = (4, 4)
mindist = ((1.5, 1.5),
           (1.5, 1.5))
n_crsov = 5    # number of structures to be generated by crossover
id_start = len(init_struc_data)  # next Structure ID
#id_start = 10

In [ ]:
co_children, co_parents, co_operation = crossover.gen_crossover(
    atype=atype,
    nat=nat,
    mindist=mindist,
    struc_data=struc_data,
    sp=sp,
    n_crsov=n_crsov,
    id_start=id_start,
)

In [ ]:
print(co_children.keys())
co_children    # {id: Structure, ...}

In [ ]:
co_parents   # to print in jupyter

In [ ]:
co_operation    # to print in jupyter

Generate children by permutation

In [ ]:
n_perm = 5    # number of structures to be generated by permutation
id_start = len(init_struc_data) + n_crsov   # next Structure ID
#id_start = 15
ntimes = 1    # number of times to perform permutation

In [ ]:
pm_children, pm_parents, pm_operation = permutation.gen_permutation(
    atype=atype,
    mindist=mindist,
    struc_data=struc_data,
    sp=sp,
    n_perm=n_perm,
    id_start=id_start,
    ntimes=ntimes,
)

In [ ]:
print(pm_children.keys())
pm_children    # {id: Structure, ...}

In [ ]:
pm_parents    # to print in jupyter

In [ ]:
pm_operation    # to print in jupyter

Generate children by strain

In [ ]:
n_strain = 5    # number of structures to be generated by strain
id_start = len(init_struc_data) + n_crsov + n_perm   # next Structure ID
#id_start = 20
sigma_st = 0.05    # standard deviation of strain

In [ ]:
st_children, st_parents, st_operation = strain.gen_strain(
    atype=atype,
    mindist=mindist,
    struc_data=struc_data,
    sp=sp,
    n_strain=n_strain,
    id_start=id_start,
    sigma_st=sigma_st,
)

In [ ]:
print(st_children.keys())
st_children    # {id: Structure, ...}

In [ ]:
st_parents    # to print in jupyter

In [ ]:
st_operation    # to print in jupyter